# Extraction des données monenfant.fr

L'extraction se fait manuellement à partir de la [recherche du site de monenfant.fr](https://monenfant.fr/que-recherchez-vous/).

Il faut :

* ouvrir préalablemet la console du navigateur sur la page "network",
* sélectionner une catégorie (e.g. "Mode d'accueil") puis une typologie (e.g. Crêches),
* remplir le captcha,
* indiquer une ville et augmenter le rayon de recherche (30km),
* lancer la recherche,
* dans la console du navigateur, repérer l'appel à la ressource `que-recherchez-vous` qui contient les résultats au format json (généralement le dernier et le plus gros),
* sauvegarder la réponse au format json dans le dossier `data` de ce dossier.

La recherche :

* se fait autour d'une ville, dans un rayon de 30km maximum,
* est limitée à 2000 résultats.

Donc, pour récupérer les données pour un département:

* faire plusieurs recherches pour des villes qui permettent de recouvrir le département,
* fusionner les résultats,
* filtrer les résultats pour exclure les données qui ne sont pas dans le département visé.


| département  | villes à rechercher                          |
|--------------|----------------------------------------------|
| Essonne      | Etampes, Arpajon                             |
| Aude         | Bram, Lagrasse, Peyriac-de-Mer, Quillan       |
| Drôme        | Peyrins, Bouvante, Crest, Taulignan, Séderon |
| Haute-Savoie | Annecy, Onnion, Passy                        |

In [ ]:
import json
from pathlib import Path

import pandas as pd


In [ ]:
import requests
import time
from datetime import datetime

now = datetime.now()
today = now.strftime("%d/%m/%Y")


def get_creche(row) -> dict:
    url = "https://monenfant.fr/web/guest/que-recherchez-vous?p_p_id=fr_monenfant_recherche_portlet_RecherchePortlet_INSTANCE_VnedXuapLnSM&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_resource_id=%2Frecherche%2Frechercher&p_p_cacheability=cacheLevelPage&_fr_monenfant_recherche_portlet_RecherchePortlet_INSTANCE_VnedXuapLnSM_cmd=get_structure_details"

    headers = {
        "User-Agent": "data.inclusion@beta.gouv.fr",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Accept-Language": "fr,en-US;q=0.7,en;q=0.3",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Requested-With": "XMLHttpRequest",
        "Origin": "https://monenfant.fr",
        "DNT": "1",
        "Connection": "keep-alive",
        "Referer": "https://monenfant.fr/que-recherchez-vous/mode-d-accueil",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
    }

    data = {
        "_fr_monenfant_recherche_portlet_RecherchePortlet_INSTANCE_VnedXuapLnSM_id": row.resultId,
        "_fr_monenfant_recherche_portlet_RecherchePortlet_INSTANCE_VnedXuapLnSM_dureeRecherche": "345",
        "_fr_monenfant_recherche_portlet_RecherchePortlet_INSTANCE_VnedXuapLnSM_dateDebutRecherche": today,
    }

    time.sleep(0.4)

    return requests.post(url, headers=headers, data=data).json()


In [ ]:
df = pd.concat(
    [
        pd.DataFrame(json.load(file.open())["results"])
        for file in Path("./data").glob("*.json")
    ],
    ignore_index=True,
)

df = df[
    df.adresse.str.contains(r"91\d{3}")  # Essonne
    | df.adresse.str.contains(r"11\d{3}")  # Aude
    | df.adresse.str.contains(r"74\d{3}")  # Drôme
    | df.adresse.str.contains(r"26\d{3}")  # Haute-Savoie
]

df = df.drop_duplicates(subset=["resultId"])

df.shape[0]


In [ ]:
df = df.apply(get_creche, axis=1, result_type="expand")

In [ ]:
df.to_json(
    "creches.json",
    orient="records",
    force_ascii=False,
)
